# 1. Setup Mario

In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [2]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [4]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000): 
    # Start the game to begin with 
    if done: 
        # Start the gamee
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


# 2. Preprocess Environment

In [5]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [6]:
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]

In [7]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [8]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [9]:
state = env.reset()

In [10]:
state, reward, done, info = env.step([5])

# 3. Train the RL Model

In [11]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [12]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [13]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [14]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [16]:
# This is the AI model started WITH SEED 42
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=42,
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [17]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/PPO_1


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 36  |
|    iterations      | 1   |
|    time_elapsed    | 14  |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 50            |
|    iterations           | 2             |
|    time_elapsed         | 20            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 6.8580266e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00272      |
|    learning_rate        | 1e-06         |
|    loss                 | 205           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000254     |
|    value_loss           | 427           |
-------------------------------------------
-----

-------------------------------------------
| time/                   |               |
|    fps                  | 86            |
|    iterations           | 13            |
|    time_elapsed         | 77            |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 2.1187006e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | -0.000603     |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0841        |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000233     |
|    value_loss           | 0.186         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 86            |
|    iterations           | 14            |
|    time_elapsed         | 82  

----------------------------------------
| time/                   |            |
|    fps                  | 90         |
|    iterations           | 24         |
|    time_elapsed         | 135        |
|    total_timesteps      | 12288      |
| train/                  |            |
|    approx_kl            | 3.3102e-05 |
|    clip_fraction        | 0          |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.94      |
|    explained_variance   | -0.00194   |
|    learning_rate        | 1e-06      |
|    loss                 | 0.17       |
|    n_updates            | 230        |
|    policy_gradient_loss | -0.000386  |
|    value_loss           | 0.357      |
----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 91           |
|    iterations           | 25           |
|    time_elapsed         | 140          |
|    total_timesteps      | 12800        |
| tr

...

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 343          |
|    time_elapsed         | 1836         |
|    total_timesteps      | 175616       |
| train/                  |              |
|    approx_kl            | 0.0009929999 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.65        |
|    explained_variance   | -0.0619      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0672       |
|    n_updates            | 3420         |
|    policy_gradient_loss | -0.00307     |
|    value_loss           | 0.132        |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 95          |
|    iterations           | 344         |
|    time_elapsed         | 1842        |
|    total_times

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 354           |
|    time_elapsed         | 1895          |
|    total_timesteps      | 181248        |
| train/                  |               |
|    approx_kl            | 0.00046333869 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.63         |
|    explained_variance   | 0.0846        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0599        |
|    n_updates            | 3530          |
|    policy_gradient_loss | -0.00109      |
|    value_loss           | 0.139         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 355          |
|    time_elapsed         | 1900    

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 365           |
|    time_elapsed         | 1953          |
|    total_timesteps      | 186880        |
| train/                  |               |
|    approx_kl            | 0.00061703054 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.59         |
|    explained_variance   | -0.0563       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.149         |
|    n_updates            | 3640          |
|    policy_gradient_loss | -0.00118      |
|    value_loss           | 0.495         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 366           |
|    time_elapsed         | 1959

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 376          |
|    time_elapsed         | 2012         |
|    total_timesteps      | 192512       |
| train/                  |              |
|    approx_kl            | 0.0003392495 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.59        |
|    explained_variance   | -0.0763      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0742       |
|    n_updates            | 3750         |
|    policy_gradient_loss | -0.000395    |
|    value_loss           | 0.131        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 377          |
|    time_elapsed         | 2018         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 387          |
|    time_elapsed         | 2071         |
|    total_timesteps      | 198144       |
| train/                  |              |
|    approx_kl            | 0.0008576632 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.49        |
|    explained_variance   | -0.0471      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.102        |
|    n_updates            | 3860         |
|    policy_gradient_loss | -0.0019      |
|    value_loss           | 0.134        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 388           |
|    time_elapsed         | 2076          |
|    t

In [20]:
model.save('thisisatestseed42model')

# 4. Test it Out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [21]:
# This is the AI model started WITH SEED 64
model64=PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=64,
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [24]:
model64.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/PPO_2
----------------------------
| time/              |     |
|    fps             | 179 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 125           |
|    iterations           | 2             |
|    time_elapsed         | 8             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 1.7021317e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00167      |
|    learning_rate        | 1e-06         |
|    loss                 | 107           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000213     |
|    value_loss           | 245           |
-------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 99            |
|    iterations           | 13            |
|    time_elapsed         | 67            |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 1.2804405e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00755      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.14          |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.00018      |
|    value_loss           | 0.393         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 98           |
|    iterations           | 14           |
|    time_elapsed         | 72      

-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 24            |
|    time_elapsed         | 126           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 2.8675771e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0711        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.138         |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.000485     |
|    value_loss           | 0.472         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 25            |
|    time_elapsed         | 132 

-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 35            |
|    time_elapsed         | 186           |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 1.6371021e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | -0.0307       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.107         |
|    n_updates            | 340           |
|    policy_gradient_loss | -2.17e-05     |
|    value_loss           | 0.486         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 96           |
|    iterations           | 36           |
|    time_elapsed         | 191     

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 46           |
|    time_elapsed         | 246          |
|    total_timesteps      | 23552        |
| train/                  |              |
|    approx_kl            | 6.992521e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.93        |
|    explained_variance   | 0.00688      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0783       |
|    n_updates            | 450          |
|    policy_gradient_loss | -0.000569    |
|    value_loss           | 0.138        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 47           |
|    time_elapsed         | 252          |
|    total_

...

------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 343          |
|    time_elapsed         | 1855         |
|    total_timesteps      | 175616       |
| train/                  |              |
|    approx_kl            | 0.0017697248 |
|    clip_fraction        | 0.00332      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.58        |
|    explained_variance   | 0.715        |
|    learning_rate        | 1e-06        |
|    loss                 | 105          |
|    n_updates            | 3420         |
|    policy_gradient_loss | -0.00131     |
|    value_loss           | 264          |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 344           |
|    time_elapsed         | 1860          |
|    t

------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 354          |
|    time_elapsed         | 1914         |
|    total_timesteps      | 181248       |
| train/                  |              |
|    approx_kl            | 0.0002784935 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.6         |
|    explained_variance   | -0.0761      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0453       |
|    n_updates            | 3530         |
|    policy_gradient_loss | -0.000825    |
|    value_loss           | 0.108        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 355          |
|    time_elapsed         | 1919         |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 365           |
|    time_elapsed         | 1973          |
|    total_timesteps      | 186880        |
| train/                  |               |
|    approx_kl            | 0.00042249786 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.6          |
|    explained_variance   | 0.0398        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0516        |
|    n_updates            | 3640          |
|    policy_gradient_loss | -0.000927     |
|    value_loss           | 0.151         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 366           |
|    time_elapsed         | 1978

-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 376           |
|    time_elapsed         | 2032          |
|    total_timesteps      | 192512        |
| train/                  |               |
|    approx_kl            | 0.00042995554 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.58         |
|    explained_variance   | -0.0229       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.119         |
|    n_updates            | 3750          |
|    policy_gradient_loss | -0.00122      |
|    value_loss           | 0.634         |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 94          |
|    iterations           | 377         |
|    time_elapsed         | 2038        

-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 387           |
|    time_elapsed         | 2091          |
|    total_timesteps      | 198144        |
| train/                  |               |
|    approx_kl            | 0.00040234486 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.57         |
|    explained_variance   | -0.132        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0857        |
|    n_updates            | 3860          |
|    policy_gradient_loss | -0.00107      |
|    value_loss           | 0.134         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 388          |
|    time_elapsed         | 2096    

In [25]:
model64.save('thisisatestseed64model')

In [26]:
# This is the AI model started WITH random seed
modelrandom=PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [27]:
modelrandom.learn(total_timesteps=200000, callback=callback)


Logging to ./logs/PPO_3
----------------------------
| time/              |     |
|    fps             | 180 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 124          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 3.420119e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | -0.00156     |
|    learning_rate        | 1e-06        |
|    loss                 | 215          |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00013     |
|    value_loss           | 499          |
------------------------------------------

------------------------------------------
| time/                   |              |
|    fps                  | 99           |
|    iterations           | 13           |
|    time_elapsed         | 67           |
|    total_timesteps      | 6656         |
| train/                  |              |
|    approx_kl            | 1.116551e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.94        |
|    explained_variance   | 0.00503      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.127        |
|    n_updates            | 120          |
|    policy_gradient_loss | -0.00015     |
|    value_loss           | 0.24         |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 98            |
|    iterations           | 14            |
|    time_elapsed         | 72            |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 24            |
|    time_elapsed         | 126           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 1.9186526e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | 0.000252      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.14          |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.000228     |
|    value_loss           | 0.432         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 25            |
|    time_elapsed         | 132 

-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 35            |
|    time_elapsed         | 185           |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 2.8474838e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | -0.0403       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.155         |
|    n_updates            | 340           |
|    policy_gradient_loss | 0.000124      |
|    value_loss           | 2.1           |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 96           |
|    iterations           | 36           |
|    time_elapsed         | 191     

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 46            |
|    time_elapsed         | 245           |
|    total_timesteps      | 23552         |
| train/                  |               |
|    approx_kl            | 0.00014528749 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0.0223        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0559        |
|    n_updates            | 450           |
|    policy_gradient_loss | -0.00127      |
|    value_loss           | 0.141         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 47            |
|    time_elapsed         | 251 

...

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 343          |
|    time_elapsed         | 1842         |
|    total_timesteps      | 175616       |
| train/                  |              |
|    approx_kl            | 0.0007799211 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.67        |
|    explained_variance   | -0.0353      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0478       |
|    n_updates            | 3420         |
|    policy_gradient_loss | -0.00182     |
|    value_loss           | 0.197        |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 95          |
|    iterations           | 344         |
|    time_elapsed         | 1848        |
|    total_times

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 354           |
|    time_elapsed         | 1902          |
|    total_timesteps      | 181248        |
| train/                  |               |
|    approx_kl            | 0.00014275836 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.67         |
|    explained_variance   | 0.0568        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0427        |
|    n_updates            | 3530          |
|    policy_gradient_loss | -0.00011      |
|    value_loss           | 0.115         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 355           |
|    time_elapsed         | 1907

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 365           |
|    time_elapsed         | 1961          |
|    total_timesteps      | 186880        |
| train/                  |               |
|    approx_kl            | 0.00013053615 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.6          |
|    explained_variance   | -0.0405       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0604        |
|    n_updates            | 3640          |
|    policy_gradient_loss | -0.000777     |
|    value_loss           | 1.3           |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 366          |
|    time_elapsed         | 1966    

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 376           |
|    time_elapsed         | 2020          |
|    total_timesteps      | 192512        |
| train/                  |               |
|    approx_kl            | 0.00012703682 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.64         |
|    explained_variance   | 0.726         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.401         |
|    n_updates            | 3750          |
|    policy_gradient_loss | -0.000386     |
|    value_loss           | 0.604         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 377           |
|    time_elapsed         | 2026

------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 387          |
|    time_elapsed         | 2079         |
|    total_timesteps      | 198144       |
| train/                  |              |
|    approx_kl            | 0.0004460836 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.62        |
|    explained_variance   | -0.0723      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0755       |
|    n_updates            | 3860         |
|    policy_gradient_loss | -0.000848    |
|    value_loss           | 0.386        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 95           |
|    iterations           | 388          |
|    time_elapsed         | 2084         |
|    total_

In [28]:
modelrandom.save('thisisatestmodel')

In [29]:
#PPO WITH MLP POLICY SEED 42
modelmlp42=PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=42,
            n_steps=512) 


Using cuda device
Wrapping the env in a VecTransposeImage.


In [30]:
modelmlp42.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/PPO_4
----------------------------
| time/              |     |
|    fps             | 243 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 145           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 2.7050264e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00335      |
|    learning_rate        | 1e-06         |
|    loss                 | 353           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000105     |
|    value_loss           | 490           |
-------------------------

------------------------------------------
| time/                   |              |
|    fps                  | 109          |
|    iterations           | 13           |
|    time_elapsed         | 61           |
|    total_timesteps      | 6656         |
| train/                  |              |
|    approx_kl            | 4.569418e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | -0.00235     |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0992       |
|    n_updates            | 120          |
|    policy_gradient_loss | -0.000132    |
|    value_loss           | 0.198        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 108           |
|    iterations           | 14            |
|    time_elapsed         | 65            |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 105           |
|    iterations           | 24            |
|    time_elapsed         | 116           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 3.5844278e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.000133      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.198         |
|    n_updates            | 230           |
|    policy_gradient_loss | -4.75e-05     |
|    value_loss           | 0.445         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 105          |
|    iterations           | 25           |
|    time_elapsed         | 121     

------------------------------------------
| time/                   |              |
|    fps                  | 104          |
|    iterations           | 35           |
|    time_elapsed         | 171          |
|    total_timesteps      | 17920        |
| train/                  |              |
|    approx_kl            | 2.155779e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | -0.000115    |
|    learning_rate        | 1e-06        |
|    loss                 | 0.137        |
|    n_updates            | 340          |
|    policy_gradient_loss | -9.63e-05    |
|    value_loss           | 0.245        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 104           |
|    iterations           | 36            |
|    time_elapsed         | 176           |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 46            |
|    time_elapsed         | 226           |
|    total_timesteps      | 23552         |
| train/                  |               |
|    approx_kl            | 2.0068837e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0001        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0986        |
|    n_updates            | 450           |
|    policy_gradient_loss | -7.32e-05     |
|    value_loss           | 0.193         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 47            |
|    time_elapsed         | 231 

...

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 343           |
|    time_elapsed         | 1702          |
|    total_timesteps      | 175616        |
| train/                  |               |
|    approx_kl            | 1.9216444e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0             |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0983        |
|    n_updates            | 3420          |
|    policy_gradient_loss | -0.000303     |
|    value_loss           | 0.176         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 103          |
|    iterations           | 344          |
|    time_elapsed         | 1707    

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 354           |
|    time_elapsed         | 1757          |
|    total_timesteps      | 181248        |
| train/                  |               |
|    approx_kl            | 1.4067627e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 9.06e-05      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0969        |
|    n_updates            | 3530          |
|    policy_gradient_loss | -6.14e-05     |
|    value_loss           | 0.217         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 355           |
|    time_elapsed         | 1762

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 365           |
|    time_elapsed         | 1811          |
|    total_timesteps      | 186880        |
| train/                  |               |
|    approx_kl            | 2.0444859e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 5.96e-08      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.136         |
|    n_updates            | 3640          |
|    policy_gradient_loss | -0.000101     |
|    value_loss           | 0.266         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 366           |
|    time_elapsed         | 1816

--------------------------------------------
| time/                   |                |
|    fps                  | 103            |
|    iterations           | 376            |
|    time_elapsed         | 1867           |
|    total_timesteps      | 192512         |
| train/                  |                |
|    approx_kl            | 1.10423425e-05 |
|    clip_fraction        | 0              |
|    clip_range           | 0.2            |
|    entropy_loss         | -1.92          |
|    explained_variance   | -6.44e-06      |
|    learning_rate        | 1e-06          |
|    loss                 | 0.0653         |
|    n_updates            | 3750           |
|    policy_gradient_loss | -0.000486      |
|    value_loss           | 0.166          |
--------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 377           |
|    time_elap

------------------------------------------
| time/                   |              |
|    fps                  | 103          |
|    iterations           | 387          |
|    time_elapsed         | 1921         |
|    total_timesteps      | 198144       |
| train/                  |              |
|    approx_kl            | 1.384411e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.92        |
|    explained_variance   | 2.92e-05     |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0496       |
|    n_updates            | 3860         |
|    policy_gradient_loss | -6.77e-05    |
|    value_loss           | 0.152        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 388           |
|    time_elapsed         | 1926          |
|    t

In [31]:
modelmlp42.save('thisisatestmlpseed42model')

In [32]:
#PPO WITH MLP POLICY SEED 64
modelmlp64=PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=64,
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [33]:
modelmlp64.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/PPO_5
----------------------------
| time/              |     |
|    fps             | 202 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 136           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 5.8825826e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00631      |
|    learning_rate        | 1e-06         |
|    loss                 | 129           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000122     |
|    value_loss           | 230           |
-------------------------

--------------------------------------------
| time/                   |                |
|    fps                  | 106            |
|    iterations           | 13             |
|    time_elapsed         | 62             |
|    total_timesteps      | 6656           |
| train/                  |                |
|    approx_kl            | 1.43776415e-05 |
|    clip_fraction        | 0              |
|    clip_range           | 0.2            |
|    entropy_loss         | -1.95          |
|    explained_variance   | 0.000575       |
|    learning_rate        | 1e-06          |
|    loss                 | 0.121          |
|    n_updates            | 120            |
|    policy_gradient_loss | -0.000167      |
|    value_loss           | 0.353          |
--------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 106          |
|    iterations           | 14           |
|    time_elapsed 

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 24            |
|    time_elapsed         | 118           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 1.9519357e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 6.56e-07      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.173         |
|    n_updates            | 230           |
|    policy_gradient_loss | -7.96e-05     |
|    value_loss           | 0.4           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 25            |
|    time_elapsed         | 123 

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 35            |
|    time_elapsed         | 172           |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 1.1331867e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -1.31e-06     |
|    learning_rate        | 1e-06         |
|    loss                 | 0.14          |
|    n_updates            | 340           |
|    policy_gradient_loss | -5.7e-05      |
|    value_loss           | 0.246         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 36            |
|    time_elapsed         | 177 

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 46            |
|    time_elapsed         | 228           |
|    total_timesteps      | 23552         |
| train/                  |               |
|    approx_kl            | 5.3661643e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | 1.34e-05      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0649        |
|    n_updates            | 450           |
|    policy_gradient_loss | -0.000161     |
|    value_loss           | 0.192         |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 103         |
|    iterations           | 47          |
|    time_elapsed         | 232         

...

-------------------------------------------
| time/                   |               |
|    fps                  | 102           |
|    iterations           | 343           |
|    time_elapsed         | 1713          |
|    total_timesteps      | 175616        |
| train/                  |               |
|    approx_kl            | 7.5662974e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0             |
|    learning_rate        | 1e-06         |
|    loss                 | 0.133         |
|    n_updates            | 3420          |
|    policy_gradient_loss | -0.000123     |
|    value_loss           | 0.314         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 102          |
|    iterations           | 344          |
|    time_elapsed         | 1718    

------------------------------------------
| time/                   |              |
|    fps                  | 102          |
|    iterations           | 354          |
|    time_elapsed         | 1769         |
|    total_timesteps      | 181248       |
| train/                  |              |
|    approx_kl            | 5.390728e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.93        |
|    explained_variance   | -7.51e-06    |
|    learning_rate        | 1e-06        |
|    loss                 | 0.107        |
|    n_updates            | 3530         |
|    policy_gradient_loss | -5.5e-05     |
|    value_loss           | 0.244        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 102          |
|    iterations           | 355          |
|    time_elapsed         | 1774         |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 102           |
|    iterations           | 365           |
|    time_elapsed         | 1824          |
|    total_timesteps      | 186880        |
| train/                  |               |
|    approx_kl            | 1.5698606e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0.000147      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.161         |
|    n_updates            | 3640          |
|    policy_gradient_loss | -0.000273     |
|    value_loss           | 0.33          |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 102           |
|    iterations           | 366           |
|    time_elapsed         | 1828

-------------------------------------------
| time/                   |               |
|    fps                  | 102           |
|    iterations           | 376           |
|    time_elapsed         | 1879          |
|    total_timesteps      | 192512        |
| train/                  |               |
|    approx_kl            | 3.6577694e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | -3.58e-07     |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0886        |
|    n_updates            | 3750          |
|    policy_gradient_loss | -0.000418     |
|    value_loss           | 0.162         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 102          |
|    iterations           | 377          |
|    time_elapsed         | 1884    

------------------------------------------
| time/                   |              |
|    fps                  | 102          |
|    iterations           | 387          |
|    time_elapsed         | 1934         |
|    total_timesteps      | 198144       |
| train/                  |              |
|    approx_kl            | 9.555777e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.93        |
|    explained_variance   | -0.00232     |
|    learning_rate        | 1e-06        |
|    loss                 | 0.069        |
|    n_updates            | 3860         |
|    policy_gradient_loss | -0.00058     |
|    value_loss           | 0.162        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 102           |
|    iterations           | 388           |
|    time_elapsed         | 1939          |
|    t

In [34]:
modelmlp64.save('thisisatestmlpseed64model')

In [35]:
#PPO WITH MLP POLICY RANDOM SEED 
modelmlprandom=PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
            n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [36]:
modelmlprandom.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/PPO_6
----------------------------
| time/              |     |
|    fps             | 238 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 146           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 3.0618394e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00694      |
|    learning_rate        | 1e-06         |
|    loss                 | 291           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000107     |
|    value_loss           | 531           |
-------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 108           |
|    iterations           | 13            |
|    time_elapsed         | 61            |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 3.2264506e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -4.68e-05     |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0922        |
|    n_updates            | 120           |
|    policy_gradient_loss | -4.45e-05     |
|    value_loss           | 0.21          |
-------------------------------------------
--------------------------------------------
| time/                   |                |
|    fps                  | 108            |
|    iterations           | 14             |
|    time_elapsed         | 

-------------------------------------------
| time/                   |               |
|    fps                  | 104           |
|    iterations           | 24            |
|    time_elapsed         | 117           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 1.0353047e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.000424     |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0992        |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.000186     |
|    value_loss           | 0.264         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 104           |
|    iterations           | 25            |
|    time_elapsed         | 122 

------------------------------------------
| time/                   |              |
|    fps                  | 104          |
|    iterations           | 35           |
|    time_elapsed         | 171          |
|    total_timesteps      | 17920        |
| train/                  |              |
|    approx_kl            | 5.568145e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | 2.26e-06     |
|    learning_rate        | 1e-06        |
|    loss                 | 9.27         |
|    n_updates            | 340          |
|    policy_gradient_loss | -0.000167    |
|    value_loss           | 33.7         |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 104           |
|    iterations           | 36            |
|    time_elapsed         | 176           |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 46            |
|    time_elapsed         | 227           |
|    total_timesteps      | 23552         |
| train/                  |               |
|    approx_kl            | 5.9168087e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.000535      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.123         |
|    n_updates            | 450           |
|    policy_gradient_loss | -0.000128     |
|    value_loss           | 0.28          |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 103           |
|    iterations           | 47            |
|    time_elapsed         | 232 

...

-------------------------------------------
| time/                   |               |
|    fps                  | 75            |
|    iterations           | 343           |
|    time_elapsed         | 2320          |
|    total_timesteps      | 175616        |
| train/                  |               |
|    approx_kl            | 2.1024607e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0.00333       |
|    learning_rate        | 1e-06         |
|    loss                 | 3.6           |
|    n_updates            | 3420          |
|    policy_gradient_loss | -6.04e-05     |
|    value_loss           | 13.9          |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 75            |
|    iterations           | 344           |
|    time_elapsed         | 2325

-------------------------------------------
| time/                   |               |
|    fps                  | 76            |
|    iterations           | 354           |
|    time_elapsed         | 2375          |
|    total_timesteps      | 181248        |
| train/                  |               |
|    approx_kl            | 6.7282235e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 1.19e-07      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.268         |
|    n_updates            | 3530          |
|    policy_gradient_loss | -0.000121     |
|    value_loss           | 0.578         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 76            |
|    iterations           | 355           |
|    time_elapsed         | 2380

-----------------------------------------
| time/                   |             |
|    fps                  | 76          |
|    iterations           | 365         |
|    time_elapsed         | 2430        |
|    total_timesteps      | 186880      |
| train/                  |             |
|    approx_kl            | 2.76044e-06 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | 5.96e-08    |
|    learning_rate        | 1e-06       |
|    loss                 | 0.318       |
|    n_updates            | 3640        |
|    policy_gradient_loss | -0.00011    |
|    value_loss           | 0.652       |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 76           |
|    iterations           | 366          |
|    time_elapsed         | 2435         |
|    total_timesteps      | 1

------------------------------------------
| time/                   |              |
|    fps                  | 77           |
|    iterations           | 376          |
|    time_elapsed         | 2486         |
|    total_timesteps      | 192512       |
| train/                  |              |
|    approx_kl            | 1.517823e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.93        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 1e-06        |
|    loss                 | 57           |
|    n_updates            | 3750         |
|    policy_gradient_loss | -5.54e-05    |
|    value_loss           | 86.3         |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 377           |
|    time_elapsed         | 2491          |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 387           |
|    time_elapsed         | 2541          |
|    total_timesteps      | 198144        |
| train/                  |               |
|    approx_kl            | 4.4878107e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.93         |
|    explained_variance   | 0             |
|    learning_rate        | 1e-06         |
|    loss                 | 0.198         |
|    n_updates            | 3860          |
|    policy_gradient_loss | -0.000227     |
|    value_loss           | 0.433         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 78            |
|    iterations           | 388           |
|    time_elapsed         | 2546

In [37]:
modelmlprandom.save('thisisatestmlprandomseedmlpmodel')

In [38]:
#USING A2C MODEL WITH SEED 42
from stable_baselines3 import A2C
modelA2C42=A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=42,
            n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [42]:
modelA2C42.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/A2C_1
-------------------------------------
| time/                 |           |
|    fps                | 175       |
|    iterations         | 100       |
|    time_elapsed       | 292       |
|    total_timesteps    | 51200     |
| train/                |           |
|    entropy_loss       | -1.95     |
|    explained_variance | -3.29e-05 |
|    learning_rate      | 1e-06     |
|    n_updates          | 99        |
|    policy_loss        | 80.8      |
|    value_loss         | 4.3e+03   |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 174       |
|    iterations         | 200       |
|    time_elapsed       | 586       |
|    total_timesteps    | 102400    |
| train/                |           |
|    entropy_loss       | -1.95     |
|    explained_variance | -0.000122 |
|    learning_rate      | 1e-06     |
|    n_updates          | 199       |
|    policy_loss        | 

In [43]:
modelA2C42.save('thisisatesta2cseed42model')

In [44]:
#USING A2C MODEL WITH SEED 64
modelA2C64=A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=64,
            n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [45]:
modelA2C64.learn(total_timesteps=200000, callback=callback)

Logging to ./logs/A2C_2
-------------------------------------
| time/                 |           |
|    fps                | 174       |
|    iterations         | 100       |
|    time_elapsed       | 293       |
|    total_timesteps    | 51200     |
| train/                |           |
|    entropy_loss       | -1.95     |
|    explained_variance | -0.000409 |
|    learning_rate      | 1e-06     |
|    n_updates          | 99        |
|    policy_loss        | 0.00704   |
|    value_loss         | 0.337     |
-------------------------------------


KeyboardInterrupt: 

In [ ]:
modelA2C64.save('thisisatesta2cseed64model')

In [ ]:
#USING A2C MODEL WITH RANDOM SEED 
modelA2CRAN=A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=64,
            n_steps=512)

In [ ]:
modelA2CRAN.learn(total_timesteps=200000, callback=callback)

In [ ]:

modelA2CRAN.save('thisisatesta2cseedRANDOMmodel')

In [ ]:
conda install python=3.7.0